In [1]:
import jieba   # 分词包
import numpy as np
import pandas as pd
import pymongo
import tensorflow as tf
# import warnings
# warnings.filterwarnings("ignore")
# import codecs
# import matplotlib.pyplot as plt
# %matplotlib inline
# import matplotlib
# matplotlib.rcParams['figure.figsize']=(10.0,5.0)
# from wordcloud import WordCloud # 词云包

In [2]:
#从数据库中读取数据
client=pymongo.MongoClient('localhost',27017)#连接数据库
db1=client['douban']#创建新数据库
comments=db1['movie_comments']#创建数据集合
pred_comments=db1['pred_comments']

data = pd.DataFrame(list(comments.find()))
# data.to_csv("douban_comments.csv",encoding="utf_8_sig")
#data=pd.read_csv("douban_comments.csv",encoding="utf_8")

In [3]:
#将没有打分的评论和3分评论去掉
def regular_nonstar(x):
    if x == 'comment-time':
        return 'allstar00 rating'
    else:
        return x

useful_data=data[['content','star']]
useful_data['star'] = useful_data.star.apply(regular_nonstar).apply(lambda x: int(x[7]))
remove0=useful_data[useful_data['star']!=0]
remove03=remove0[remove0['star']!=3]
remove03.head(2)

D:\Program\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,content,star
0,当然算不上杰作，但放豆瓣语境下，是部时至今日终于拍出来的国产“高分韩国电影”。拿现实题材拍商...,4
1,最大的病，其实是穷病。真的被感动了，整体都很成熟，也有些许韩片的影子。几个演员表演都非常出色...,4


In [ ]:
# 分词
def lcut(Intro_movie):
    segs = jieba.lcut(Intro_movie) # jiaba.lcut()
#     return segs
    segment=[]
    for seg in segs:
        if len(seg)>1 and seg!='\r\n':
            segment.append(seg)
    return segment

# 去停用词
def dropstopword(segment):
    words_df = pd.DataFrame({'segment':segment})
    stopwords = pd.read_csv("stopwords2.txt" 
                      ,index_col=False
                      ,quoting=3
                      ,sep="\t"
                      ,names=['stopword']
                      ,encoding='utf-8') # quoting=3 全不引用  
    words_list=words_df[~words_df.segment.isin(stopwords.stopword)].segment.values.tolist()
    
    return words_list
remove03['processed_comment']=remove03.content.apply(lcut).apply(dropstopword)
# remove03.to_csv(r"douban_comments.csv",encoding="utf_8_sig")
remove03.head(2)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\chang\AppData\Local\Temp\jieba.cache
Loading model cost 0.601 seconds.
Prefix dict has been built succesfully.


In [11]:
import json
pred_comments.insert(json.loads(remove03.T.to_json()).values())

D:\Program\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


[ObjectId('5cdbd2cda1cf18259c85b40e'),
 ObjectId('5cdbd2cda1cf18259c85b40f'),
 ObjectId('5cdbd2cda1cf18259c85b410'),
 ObjectId('5cdbd2cda1cf18259c85b411'),
 ObjectId('5cdbd2cda1cf18259c85b412'),
 ObjectId('5cdbd2cda1cf18259c85b413'),
 ObjectId('5cdbd2cda1cf18259c85b414'),
 ObjectId('5cdbd2cda1cf18259c85b415'),
 ObjectId('5cdbd2cda1cf18259c85b416'),
 ObjectId('5cdbd2cda1cf18259c85b417'),
 ObjectId('5cdbd2cda1cf18259c85b418'),
 ObjectId('5cdbd2cda1cf18259c85b419'),
 ObjectId('5cdbd2cda1cf18259c85b41a'),
 ObjectId('5cdbd2cda1cf18259c85b41b'),
 ObjectId('5cdbd2cda1cf18259c85b41c'),
 ObjectId('5cdbd2cda1cf18259c85b41d'),
 ObjectId('5cdbd2cda1cf18259c85b41e'),
 ObjectId('5cdbd2cda1cf18259c85b41f'),
 ObjectId('5cdbd2cda1cf18259c85b420'),
 ObjectId('5cdbd2cda1cf18259c85b421'),
 ObjectId('5cdbd2cda1cf18259c85b422'),
 ObjectId('5cdbd2cda1cf18259c85b423'),
 ObjectId('5cdbd2cda1cf18259c85b424'),
 ObjectId('5cdbd2cda1cf18259c85b425'),
 ObjectId('5cdbd2cda1cf18259c85b426'),
 ObjectId('5cdbd2cda1cf18

In [6]:
#读取腾讯AI Lab词向量并保存为我所需要的词向量
from gensim.models import KeyedVectors
wv_from_text = KeyedVectors.load_word2vec_format(r'F:\IDM下载\压缩文件\Tencent_AILab_ChineseEmbedding.txt', binary=False)
# new_wordvec={}
# def save_new_vec(words_list):
#     global new_wordvec
#     for i in words_list:
#         if i in wv_from_text:
#             new_wordvec[i]=wv_from_text[i]
# remove03.processed_comment.apply(save_new_vec) 
# np.save('sample_Tencent_AILab_ChineseEmbedding.npy', new_wordvec) 
# #word_vectors=np.load('sample_Tencent_AILab_ChineseEmbedding.npy').item()

In [ ]:
word_vectors=np.load('sample_Tencent_AILab_ChineseEmbedding.npy').item()
#转换词向量
def convert_to_vec(words_list):
    words_vec=[]
    for i in word_vectors:
        if i in wv_from_text:
            words_vec.append(wv_from_text[i])
    return words_vec

# 转换评星
def convert_stars(star):
    if star>3:
        return 1
    else:
        return 0

remove03['word_vec']=remove03.processed_comment.apply(convert_to_vec)
remove03['pos1neg0']=remove03.star.apply(convert_stars)
processed_data=remove03
#processed_data.to_csv(r"douban_comments_processed.csv",encoding="utf_8_sig")
#pd.read_csv("douban_comments_processed.csv",encoding="utf_8")
processed_data.head(2)

In [9]:
num2=np.mean(processed_data[:].processed_comment.apply(lambda x:len(x)))
num1=np.mean(processed_data[:].word_vec.apply(lambda x:len(x)))

print('平均每个评论有',int(num2),'个词')
print('平均每个评论有',int(num1),'个 wordvec')

平均每个评论有 16 个词
平均每个评论有 16 个 wordvec
